In [1]:
from google.colab import files
files.upload()

Saving stocks.csv to stocks.csv
Saving trade_train.csv to trade_train.csv


In [3]:
import pandas as pd

stocks = pd.read_csv('stocks.csv')
trade = pd.read_csv('trade_train.csv')

In [4]:
def Group(trade):
#      그룹번호 별 데이터를 쉽게 볼 수 있게
#     Trade_group안에 데이터를 리스트 형태로 append함
    group = []
    for i in trade['그룹번호']:
        group.append(i)
        group = sorted(list(set(group)))
    trade_group = []
    for i in group:
        trade_group.append(trade.groupby('그룹번호').get_group('{}'.format(i)))
    return trade_group

In [5]:
trade_group = Group(trade)

In [6]:
def Year_month(trade_group):
    ## 년 별 월별 데이터 기준년월에 추가
    for group in trade_group:
        group['year'] = trade_group[0]['기준년월']//100
        group['month'] = trade_group[0]['기준년월']%100
    return trade_group

In [7]:
trade_group = Year_month(trade_group)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [8]:
def each_stock_number(trade_group):
    trade_stock_list = []
    for group in trade_group:
        trade_stock_name = group['종목번호'].unique()
        trade_stock_list.append(trade_stock_name)
    return trade_stock_list

In [9]:
trade_stock_list = each_stock_number(trade_group)

In [10]:
trade_stock_list

[array(['A000660', 'A001820', 'A004020', 'A005380', 'A005930', 'A005935',
        'A006400', 'A008770', 'A009150', 'A034220', 'A066570', 'A128940',
        'A005290', 'A020150', 'A138080', 'A215600', 'A000270', 'A009540',
        'A011070', 'A061970', 'A068270', 'A028050', 'A028300', 'A033180',
        'A036830', 'A084990', 'A207940', 'A003670', 'A032500', 'A060720',
        'A010950', 'A051910', 'A123040', 'A012330', 'A012610', 'A015760',
        'A033640', 'A034230', 'A035420', 'A039130', 'A090430', 'A032640',
        'A035720', 'A036540', 'A083550', 'A096770', 'A180640', 'A247540',
        'A005490', 'A009830', 'A010130', 'A010140', 'A096530', 'A241820',
        'A253840', 'A000210', 'A000990', 'A008350', 'A011000', 'A011780',
        'A017670', 'A028260', 'A091990', 'A253450', 'A272290', 'A294140',
        'A003000', 'A018260', 'A024110', 'A030200', 'A034020', 'A036570',
        'A068760', 'A085660', 'A105560', 'A131390', 'A192820', 'A003620',
        'A006800', 'A017960', 'A020560

In [ ]:
# 각그룹별 시행

#필요 함수들 정리

In [59]:
def stock_feature(stocks, feature_list):
  # Choice stock_feature
   stock_feature = stocks[feature_list]
   stock_feature = stock_feature.sort_values(by='종목번호')
   stock_feature = stock_feature.drop_duplicates(['종목번호'])
   print(stock_feature.head())
   return stock_feature

def Trade_top3_1(trade_group):
  # 매수고객수 sort_values
  df = pd.DataFrame(trade_group.groupby(['종목번호'])['매수고객수'].sum())
  df = df.sort_values(by = ['매수고객수'], ascending = False).head(3)
  return pd.DataFrame(df)

def Trade_top3_Date(trade_group, stock_feature):
  df2 = trade_group.sort_values(by="매수고객수", ascending=False).groupby("기준년월").head(3)  #'기준년월' 기준으로 grouping 한뒤 매수고객수가 가장 많은 종목 top3
  df2 = df2.sort_values(['기준년월', '매수고객수'], ascending=[True, False])
  df2 = df2.reset_index(drop=True)                #다시 인덱싱
  df2 = df2[['기준년월','종목번호','매수고객수']] #필요 column만 추출
  df2 = pd.merge(df2, stock_feature, on="종목번호", how="left")  #각 종목 특성 가져옴
  return df2

# def Trade_top3_1(trade_stock_list, trade_group):
#             Total_list = []
#             for source in trade_stock_list:
#                 total = 0
#                 lab = []
#                 for k in trade_group[trade_group['종목번호']=='{}'.format(source)]['매수고객수']:
#                     total += k
#                 lab.append(source)
#                 lab.append(total)
#                 Total_list.append(lab)
#             return Total_list

In [60]:
stock_feature =  stock_feature(stocks, ['종목번호','종목명','20년7월TOP3대상여부','시장구분','표준산업구분코드_대분류','표준산업구분코드_중분류','표준산업구분코드_소분류','거래량','거래금액_만원단위'])

           종목번호       종목명 20년7월TOP3대상여부  ... 표준산업구분코드_소분류     거래량 거래금액_만원단위
0       A000020      동화약품             N  ...      의약품 제조업   31341     30581
119392  A000080     하이트진로             N  ...    알코올음료 제조업  914681   2488133
127991  A000100      유한양행             Y  ...      의약품 제조업   27837    681620
235021  A000120    CJ대한통운             N  ...    도로 화물 운송업   60144    949139
18520   A000140  하이트진로홀딩스             Y  ...       기타 금융업  129586    124078

[5 rows x 9 columns]


In [61]:
type(stock_feature)

pandas.core.frame.DataFrame

In [62]:
Trade_top3_list = []
for i in trade_group: # i = 각 그룹별 매수 종목번호
    Trade_top3 = Trade_top3_1(i)
    Trade_top3 = pd.merge(Trade_top3,stock_feature, on = '종목번호', how = 'left')
    Trade_top3_list.append(Trade_top3)

In [63]:
Trade_top3_Date_list = []
for i in trade_group:
  Trade_top3_Date_list.append(Trade_top3_Date(i,stock_feature))

In [67]:
Trade_top3_Date_list[1]

# 월별 Top3 데이터를 그룹별 리스트화 해서 정리함.

,기준년월,종목번호,매수고객수,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,거래량,거래금액_만원단위
0,201907,A042670,4,두산인프라코어,Y,코스피,제조업,기타 기계 및 장비 제조업,특수 목적용 기계 제조업,677430,424256
1,201907,A006800,3,미래에셋대우,Y,코스피,금융 및 보험업,금융 및 보험 관련 서비스업,금융 지원 서비스업,860220,620650
2,201907,A035420,0,NAVER,Y,코스피,정보통신업,정보서비스업,자료처리· 호스팅· 포털 및 기타 인터넷 정보매개 서비스업,246407,3821975
3,201908,A215600,4,신라젠,N,코스닥,전문· 과학 및 기술 서비스업,연구개발업,자연과학 및 공학 연구개발업,1601301,1797372
4,201908,A005930,4,삼성전자,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,21157172,112943103
5,201908,A000660,3,SK하이닉스,Y,코스피,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업,5831965,50901766
6,201910,A005930,4,삼성전자,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,21157172,112943103
7,201910,A215600,4,신라젠,N,코스닥,전문· 과학 및 기술 서비스업,연구개발업,자연과학 및 공학 연구개발업,1601301,1797372
8,201910,A028300,3,에이치엘비,N,코스닥,제조업,기타 운송장비 제조업,선박 및 보트 건조업,984387,9951609
9,201911,A005930,3,삼성전자,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,21157172,112943103


In [ ]:
# def None_in_stocks(trade_stock_list, stocks):
#     None_in_stocks = []
#     for i in trade_stock_list:
#         if len(stocks[stocks['종목전호']== i]) == 0:
#             print(i)
#             None_in_stocks.append(i)
#     return None_in_stocks

In [ ]:
# def Trade_top3_2(trade_stock_list, stocks):
#     stock_sample =[]
#     stock_sample_end = []
#     for i in trade_stock_list:
#         if len(stocks[stocks['종목번호'] == i]) == 0:
#             pass
#         else:
#             stock_sample.append(stocks[stocks['종목번호']==i].index[0])                      
#     stock_sample_end.append(stock_sample)  
#     stock_sample = []
#     return stock_sample_end # 한집합. 

In [ ]:
# def Trade_top3_3(stock_sample_end,stocks,Total_list):
#     Real_total_list = []
#     stocks_group_1 = stocks.loc[stock_sample_end,:]
#     for i in Total_list:
#         for j in stocks_group_1['종목번호']:
#             if i[0] == j:
#                 Real_total_list.append(i[1])
#     stocks_group_1['매수고객수 총계'] = Real_total_list
# #     stocks_group_1 =  stocks_group_1[stocks_group_1['20년7월TOP3대상여부'] == 'Y']
#     stocks_group_1.sort_values(by=['매수고객수 총계'], axis=0, ascending=False)
#     return stocks_group_1
    

In [ ]:
# def Trade_top3_3(stock_sample_end, stocks):
#     df2 = group1.sort_values(by = "매수고객수", ascending = False_.groupby("기준년월").head(3)
#     df2.sort_values(['기준년월', '매수고객수'], ascending = [True, False])
#     df2 = df2.rest_index(drop = True)
#     df2 = df2[['기준년월', '종목번호'. '매수고객수']]
#     df2 = pd.merge(df2, stock_feature, on = "종목번호", how = "left")